# TerraClimate Downscaling - Contemporary and Climate Futures

***
<font color=red>Title: Downscale TerraClimate <i>Contemporary</i> and <i>Climate Futures</i> 2C and 4C water balance raster variables to 6-minute resolution and create a time series of discharge rasters consistent with CUNY WBM river network.
Author: Pamela A. Green  
Date: December 21, 2022  

This code hase been modified and updated from the original developed between January - December 31, 2022 by Pamela Green, Senior Research Associate, CUNY Advanced Science Research Center, New York, NY. Code has been updated to be generically applicable to any raster water balance model output although end user will need to ensure all raster data are in a consistent resolution and projection.  

The code uses an existing Water Balance Model function from the RGIS model (https://github.com/bmfekete/RGIS) to route runoff along a river network with a simple flow accumulation model. The RGIS functions are run via a command line argument using the python subprocess module. This will require the user to install RGIS on their local system via the GitHub link provided above. Alternatively, user may choose to use a different river network and/or network routing function in place of the WBM RGIS model. </font>
***

This code downscales TerraClimate <i>Contemporary</i> and <i>Climate Futures (2C,4C)</i> data from native NetCDF format at 2min30sec resolution into 6-minute GeoTIFF rasters. Gridded runoff is calculated as TerraClimate [Precipitation (PPT) - Actual Evapotranspiration (AET)] since no runoff ('q') data is provided for <i>Climate Futures</i> runs. Downscaled runoff is accumulated along CUNY WBM river networks using the RGIS netAccumulate function to create a timeseries of discharge grids consistent with WMB river topology.

The following steps were carried out to develop the datasets in this analysis:
- TerraClimate data for Contemporary, 2C and 4C climate runs in native 2m30s NetCDF format is downloaded from https://www.climatologylab.org/terraclimate.html and stored at <i>xyxy</i>
- Aggregate Monthly Data to Annual values for annual analysis
- Resolution is downscaled to 6-minute using gdal.warp
- Create Long Term Means of runoff calculated
- Flow accumulation with RGIS netAccumulate function to derive Discharge time series of rasters
    
TerraClimate multi-decadal <i>Contemporary and Climate Futures</i> forecasts for 2C and 4C runs downloaded via TerraClimate website at https://www.climatologylab.org/terraclimate.html

Qin, Y., Abatzoglou, J.T., Siebert, S. et al. Agricultural risks from changing snowmelt. Nat. Clim. Chang. 10, 459–465 (2020). https://doi.org/10.1038/s41558-020-0746-8.

# Modules and Functions

In [ ]:
import rgis as rg
import numpy as np
import pandas as pd
import scipy
import subprocess as sp

from scipy import stats
from scipy.stats.mstats import rankdata
from osgeo import gdal, gdal_array, osr
from osgeo.gdalconst import *
import osgeo.gdalnumeric as gdn
import xarray as xr

import gzip
import shutil

## Single band numpy array to GeoTiff file

In [ ]:
def save2File(rA, outname, nrows, ncols, geo_transform):  
    outDs = gdal.GetDriverByName('GTiff').Create(outname, ncols, nrows, 1, gdal.GDT_Float32)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(rA)
    outDs.SetGeoTransform(geo_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    outDs.SetProjection(srs.ExportToWkt())
    outDs = None

    """
    geotransform[0] = top left x
    geotransform[1] = w-e pixel resolution
    geotransform[2] = 0
    geotransform[3] = top left y
    geotransform[4] = 0
    geotransform[5] = n-s pixel resolution (negative value)
    """

## MultiBand numpy array to GeoTiff

In [ ]:
def CreateMultiGeoTiff(Array, Name, driver, NDV, GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    DataSet = gdal.GetDriverByName(driver).Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    DataSet.SetGeoTransform(GeoT)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(Projection)
    DataSet.SetProjection(srs.ExportToWkt() )
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

# Global Variables, Definitions, Datasets

In [ ]:
TC_outdir = '/asrc/ecr/pamela/TerraClimate/'
TC_ClimFut = '/asrc/ecr/pamela/TerraClimate/ClimateFutures/'
TC_Contemp = '/asrc/ecr/pamela/TerraClimate/raw/'

n = 300

## Explicitly define projection information for native 2min30s TerraClimate raster and 6-minute target resolution rasters

In [ ]:
ncols2m30 = 8640
nrows2m30 = 4320
xll2m30 = -180
yll2m30 = -90
xur2m30 = 180
yur2m30 = 90
cellsize2m30 = 0.041666667
cellang = cellsize2m30 * -1
nodata = -9999
geotr2m30 = ([ xll2m30, cellsize2m30, 0, yur2m30, 0, cellang ])

ncols6m = 3600
nrows6m = 1500
xll6m = -180
yll6m = -60
xur6m = 180
yur6m = 90
cellsize6m = 0.1
cellang = cellsize6m * -1
nodata = -9999
geotr6m = ([ xll6m, cellsize6m, 0, yur6m, 0, cellang ])

geodriver = 'GTiff'
geoproj = 4326

head2m30 = 'ncols ' + str(ncols2m30) + '\nnrows ' + str(nrows2m30) + '\nxllcorner ' + str(xll2m30) + '\nyllcorner ' + str(yll2m30) + '\ncellsize ' + str(cellsize2m30) + '\nNODATA_value ' + str(nodata)
head6m = 'ncols ' + str(ncols6m) + '\nnrows ' + str(nrows6m) + '\nxllcorner ' + str(xll6m) + '\nyllcorner ' + str(yll6m) + '\ncellsize ' + str(cellsize6m) + '\nNODATA_value ' + str(nodata)


## Import 6-minute CellArea raster

In [ ]:
fin = '/asrc/ecr/pamela/cellarea/CellArea_6m.tif'
ds = gdal.Open(str(fin))
cellarea6m = np.array(ds.GetRasterBand(1).ReadAsArray())

# Calculate ANNUAL TerraClimate (TC) Climate Runoff for Contemporary, 2C and 4C climate runs from PPT - AET at 2min30s resolution, Resample Annual Runoff from Native 2m30s to 6 minute and NetAccumulate

    Read netCDF into xarray (gunzip netCDF4 format file, if needed)
    Sum PPT and AET over time dimension to get annual values
    Calculate Annual Runoff = Annual PPT - Annual AET
    Output to GeoTiff and resample to 6min using gdal.Warp
    Output 6-Minute runoff to gridascii (for input to rgis and netAccumulate)
    Flow accumulate runoff to discharge using RGIS netAccumulate
    Create xarrays for monthly 6min rasters

In [ ]:
# Define Start and End Years for time series
StartYear = 1985
EndYear = 2016
EndYearTxt = EndYear - 1

## Contemporary

In [ ]:
print("Resample TC to 6min...")
    
for x in range(StartYear, EndYear):
    
    aetname = TC_Contemp + 'TerraClimate_aet_' + str(x) + '.nc'
    pptname = TC_Contemp + 'TerraClimate_ppt_' + str(x) + '.nc'
    
    print(str(x) + " ", end='')
    
    # Import TC AET and PPT into xarray
    ds_aet = xr.open_dataset(aetname)
    ds_ppt = xr.open_dataset(pptname)
    
    # Create annual TC AET and PPT from monthly xarray data
    aet_annual = ds_aet['aet'].sum('time').values
    ppt_annual = ds_ppt['ppt'].sum('time').values

    # Create annual and monthly TC runoff from PPT - AET
    q_annual = ppt_annual - aet_annual
    q_annual = np.where(q_annual < 0, 0, q_annual)
    
    # Save annnual to GeoTIFF, resample to 6min and output to gridascii for rgis flow accumulation
    tempname = TC_Contemp + 'q_annual2m30.tif'
    save2File(q_annual[:, :], tempname, nrows2m30, ncols2m30, geotr2m30)
    infn = TC_Contemp + 'q_annual2m30.tif'
    outfn = TC_Contemp + 'TerraClimate_RO_annual' + str(x) + '_6m.tif'
    xres=0.1
    yres=0.1
    resample_alg = 'average'
    options = gdal.WarpOptions(options=['tr'], xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = gdal.Warp(outfn, infn, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = None   
    ds = gdal.Open(str(outfn))
    qanntif = np.array(ds.GetRasterBand(1).ReadAsArray())
    qanntif[np.isnan(qanntif)] = 0
    qanntif = qanntif[:-n, :]
    qanntif = np.where(qanntif == -9999, 0.00, qanntif)
    qanntif_mm = qanntif
    qanntif_mm = np.resize(qanntif_mm, (1,nrows6m,ncols6m))
    
    qanntif = qanntif * cellarea6m * (0.0000316880878)
    qanntif = np.resize(qanntif, (1,nrows6m,ncols6m))
    
    if x == StartYear:
        tcrov_ann_array = qanntif
        tcro_ann_array = qanntif_mm
    else:
        tcrov_ann_array = np.concatenate(([tcrov_ann_array, qanntif]), axis=0)
        tcro_ann_array = np.concatenate(([tcro_ann_array, qanntif_mm]), axis=0)
        
    np.savetxt(TC_Contemp + 'qann.txt', qanntif[0, :, :], fmt='%1.9f', delimiter='  ', header=str(head6m), comments='')
    cmd = TC_Contemp + 'netAcc6m_annContemp.sh'
    sp.call(cmd, shell=True)
    cmd = 'mv ' + TC_Contemp + 'Discharge_ann.gdbc.gz ' + TC_Contemp + 'TerraClimate_Discharge_4C_' + str(x) + '_6m.gdbc.gz'
    sp.call(cmd, shell=True)
    
    name = str(TC_Contemp) + 'TerraClimate_Discharge_' + str(x) + '_6m.gdbc.gz'
    QANN = rg.grid(name)
    QANN.Load()
    qann = QANN.Data  # .flatten()
    qann[np.isnan(qann)] = 0

    if x == StartYear:
        qann_ann_array = qann
    else:
        qann_ann_array = np.concatenate(([qann_ann_array, qann]), axis=0)
        
    cmd_remove = 'rm -f ' + TC_Contemp + 'qann.txt'
    sp.call(cmd_remove, shell=True)

# Remove gdbc files no longer needed
cmd_remove = 'rm -f ' + TC_Contemp + '*.gdbc.gz'
sp.call(cmd_remove, shell=True)

tempname = TC_Contemp + 'TerraClimate_RO_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(tcro_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

tempname = TC_Contemp + 'TerraClimate_ROV_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(tcrov_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

tempname = TC_Contemp + 'TerraClimate_Discharge_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(qann_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)


## 4C

In [ ]:
print("Resample TC to 6min...")
    
for x in range(StartYear, EndYear):
    
    aetname = TC_ClimFut + '4C/TerraClimate_4c_aet_' + str(x) + '.nc'
    pptname = TC_ClimFut + '4C/TerraClimate_4c_ppt_' + str(x) + '.nc'
    
    print(str(x) + " ", end='')
    
    # Import TC AET and PPT into xarray
    ds_aet = xr.open_dataset(aetname)
    ds_ppt = xr.open_dataset(pptname)
    
    # Create annual TC AET and PPT from monthly xarray data
    aet_annual = ds_aet['aet'].sum('time').values
    ppt_annual = ds_ppt['ppt'].sum('time').values

    # Create annual and monthly TC runoff from PPT - AET
    q_annual = ppt_annual - aet_annual
    q_annual = np.where(q_annual < 0, 0, q_annual)
    
    # Save annnual to GeoTIFF, resample to 6min and output to gridascii for rgis flow accumulation
    tempname = TC_ClimFut + '4C/q_annual2m30.tif'
    save2File(q_annual[:, :], tempname, nrows2m30, ncols2m30, geotr2m30)
    infn = TC_ClimFut + '4C/q_annual2m30.tif'
    outfn = TC_ClimFut + '4C/TerraClimate_RO_annual' + str(x) + '_6m.tif'
    xres=0.1
    yres=0.1
    resample_alg = 'average'
    options = gdal.WarpOptions(options=['tr'], xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = gdal.Warp(outfn, infn, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = None   
    ds = gdal.Open(str(outfn))
    qanntif = np.array(ds.GetRasterBand(1).ReadAsArray())
    qanntif[np.isnan(qanntif)] = 0
    qanntif = qanntif[:-n, :]
    qanntif = np.where(qanntif == -9999, 0.00, qanntif)
    qanntif_mm = qanntif
    qanntif_mm = np.resize(qanntif_mm, (1,nrows6m,ncols6m))
    
    qanntif = qanntif * cellarea6m * (0.0000316880878)
    qanntif = np.resize(qanntif, (1,nrows6m,ncols6m))
    
    if x == StartYear:
        tcrov_ann_array = qanntif
        tcro_ann_array = qanntif_mm
    else:
        tcrov_ann_array = np.concatenate(([tcrov_ann_array, qanntif]), axis=0)
        tcro_ann_array = np.concatenate(([tcro_ann_array, qanntif_mm]), axis=0)
        
    np.savetxt(TC_ClimFut + 'qann.txt', qanntif[0, :, :], fmt='%1.9f', delimiter='  ', header=str(head6m), comments='')
    cmd = TC_ClimFut + 'netAcc6m_annClimFut.sh'
    sp.call(cmd, shell=True)
    cmd = 'mv ' + TC_ClimFut + 'Discharge_ann.gdbc.gz ' + TC_ClimFut + '4C/TerraClimate_Discharge_4C_' + str(x) + '_6m.gdbc.gz'
    sp.call(cmd, shell=True)
    
    name = str(TC_ClimFut) + '4C/TerraClimate_Discharge_4C_' + str(x) + '_6m.gdbc.gz'
    QANN = rg.grid(name)
    QANN.Load()
    qann = QANN.Data  # .flatten()
    qann[np.isnan(qann)] = 0

    if x == StartYear:
        qann_ann_array = qann
    else:
        qann_ann_array = np.concatenate(([qann_ann_array, qann]), axis=0)
        
    cmd_remove = 'rm -f ' + TC_ClimFut + 'qann.txt'
    sp.call(cmd_remove, shell=True)

# Remove gdbc files no longer needed
cmd_remove = 'rm -f ' + TC_ClimFut + '4C/*.gdbc.gz'
sp.call(cmd_remove, shell=True)

tempname = TC_ClimFut + '4C/TerraClimate_RO_4C_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(tcro_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

tempname = TC_ClimFut + '4C/TerraClimate_ROV_4C_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(tcrov_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

tempname = TC_ClimFut + '4C/TerraClimate_Discharge_4C_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(qann_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)


## 2C

In [ ]:
print("Resample TC to 6min...")
    
for x in range(StartYear, EndYear):
    
    aetname = TC_ClimFut + '2C/TerraClimate_2c_aet_' + str(x) + '.nc'
    pptname = TC_ClimFut + '2C/TerraClimate_2c_ppt_' + str(x) + '.nc'
    
    print(str(x) + " ", end='')
    
    # Import TC AET and PPT into xarray
    ds_aet = xr.open_dataset(aetname)
    ds_ppt = xr.open_dataset(pptname)
    
    # Create annual TC AET and PPT from monthly xarray data
    aet_annual = ds_aet['aet'].sum('time').values
    ppt_annual = ds_ppt['ppt'].sum('time').values

    # Create annual and monthly TC runoff from PPT - AET
    q_annual = ppt_annual - aet_annual
    q_annual = np.where(q_annual < 0, 0, q_annual)
    
    x2 = 4800
    y2 = 2160
    
    # Save annnual to GeoTIFF, resample to 6min and output to gridascii for rgis flow accumulation
    tempname = TC_ClimFut + '2C/q_annual2m30.tif'
    save2File(q_annual[:, :], tempname, nrows2m30, ncols2m30, geotr2m30)
    infn = TC_ClimFut + '2C/q_annual2m30.tif'
    outfn = TC_ClimFut + '2C/TerraClimate_RO_annual' + str(x) + '_6m.tif'
    xres=0.1
    yres=0.1
    resample_alg = 'average'
    options = gdal.WarpOptions(options=['tr'], xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = gdal.Warp(outfn, infn, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = None   
    ds = gdal.Open(str(outfn))
    qanntif = np.array(ds.GetRasterBand(1).ReadAsArray())
    qanntif[np.isnan(qanntif)] = 0
    qanntif = qanntif[:-n, :]
    qanntif = np.where(qanntif == -9999, 0.00, qanntif)
    qanntif_mm = qanntif
    qanntif_mm = np.resize(qanntif_mm, (1,nrows6m,ncols6m))
    
    qanntif = qanntif * cellarea6m * (0.0000316880878)
    qanntif = np.resize(qanntif, (1,nrows6m,ncols6m))

    if x == StartYear:
        tcrov_ann_array = qanntif
        tcro_ann_array = qanntif_mm     
    else:
        tcrov_ann_array = np.concatenate(([tcrov_ann_array, qanntif]), axis=0)
        tcro_ann_array = np.concatenate(([tcro_ann_array, qanntif_mm]), axis=0)       
        
    np.savetxt(TC_ClimFut + 'qann.txt', qanntif[0, :, :], fmt='%1.9f', delimiter='  ', header=str(head6m), comments='')
    cmd = TC_ClimFut + 'netAcc6m_annClimFut.sh'
    sp.call(cmd, shell=True)
    cmd = 'mv ' + TC_ClimFut + 'Discharge_ann.gdbc.gz ' + TC_ClimFut + '2C/TerraClimate_Discharge_2C_' + str(x) + '_6m.gdbc.gz'
    sp.call(cmd, shell=True)
    
    name = str(TC_ClimFut) + '2C/TerraClimate_Discharge_2C_' + str(x) + '_6m.gdbc.gz'
    QANN = rg.grid(name)
    QANN.Load()
    qann = QANN.Data  # .flatten()
    qann[np.isnan(qann)] = 0

    if x == StartYear:
        qann_ann_array = qann        
    else:
        qann_ann_array = np.concatenate(([qann_ann_array, qann]), axis=0)
        
    cmd_remove = 'rm -f ' + TC_ClimFut + 'qann.txt'
    sp.call(cmd_remove, shell=True)

# Remove gdbc files no longer needed
cmd_remove = 'rm -f ' + TC_ClimFut + '2C/*.gdbc.gz'
sp.call(cmd_remove, shell=True)

tempname = TC_ClimFut + '2C/TerraClimate_RO_2C_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(tcro_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

tempname = TC_ClimFut + '2C/TerraClimate_ROV_2C_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(tcrov_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

tempname = TC_ClimFut + '2C/TerraClimate_Discharge_2C_Ann' + str(StartYear) + '-' + str(EndYearTxt) + '.tif'
CreateMultiGeoTiff(qann_ann_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)


# Calculate MONTHLY TerraClimate (TC) Climate Runs for 2C and 4C climate runs from PPT - AET at 2min30s resolution, Resample Annual Runoff from Native 2m30s to 6 minute and NetAccumulate

    Read netCDF into xarray (gunzip netCDF4 format file)
    Calculate Monthly Runoff = Monthly PPT - Monthly AET
    Output to GeoTiff and resample to 6min using gdal.Warp
    Output 6-Minute runoff to gridascii (for input to rgis and netAccumulate)
    Flow accumulate runoff to discharge using RGIS netAccumulate
    Create xarrays for monthly 6-minute rasters

## Contemporary runs

In [ ]:
print("Resample TC to 6min...")
    
for x in range(StartYear, EndYear):
        
    # Calculate TC runoff from monthly AET and PPT xarrrays and save to GeoTIFFs for resampling
    aet_mo = ds_aet['aet'].values
    ppt_mo = ds_ppt['ppt'].values
    qmo = ppt_mo - aet_mo
    qmo = np.where(qmo < 0, 0, qmo)
    tempname = TC_Contemp + 'q_mo2m30.tif'
    CreateMultiGeoTiff(qmo[:, :, :], tempname, geodriver, nodata, geotr2m30, geoproj, gdal.GDT_Float32)
    
    # Resample 2m30s monthly rasters to 6m resolution with nearest neighbor
    infn = TC_Contemp + 'q_mo2m30.tif'
    outfn = TC_Contemp + 'TerraClimate_RO_' + str(x) + '_6m.tif'
    xres=0.1
    yres=0.1
    resample_alg = 'average'
    options = gdal.WarpOptions(options=['tr'], xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = gdal.Warp(outfn, infn, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = None
    
    ds = gdal.Open(str(outfn))
    #qannmo = np.array(ds.GetRasterBand(1).ReadAsArray())
    qannmo = ds.ReadAsArray()
    qannmo[np.isnan(qannmo)] = 0
    qannmo = qannmo[:, :-n, :]
    qannmo = np.where(qannmo < 0, 0.00, qannmo)
    
    cellarea6m3D = np.resize(cellarea6m, (1,nrows6m,ncols6m))
    qannmo = qannmo * cellarea6m3D * (0.000380257053768347)
    
    if x == StartYear:
        tcrov_mo_array = qannmo
    else:
        tcrov_mo_array = np.concatenate(([tcrov_mo_array, qannmo]), axis=0)

    for y in range(0, 12):
        month = y + 1
        print(str(x) + "-" + str(month) + " ", end='')
        np.savetxt(TC_Contemp + 'qmo.txt', qannmo[y, :, :], fmt='%1.9f', delimiter='  ', header=str(head6m), comments='')
        cmd = TC_Contemp + 'netAcc6m_moClimFut.sh'
        sp.call(cmd, shell=True)
        cmd = 'mv ' + TC_Contemp + 'Discharge_mo.gdbc.gz ' + TC_Contemp + 'TerraClimate_Discharge_' + str(x) + '-' + str(month) + '_6m.gdbc.gz'
        sp.call(cmd, shell=True)
    
        name = str(TC_Contemp) + 'TerraClimate_Discharge_' + str(x) + '-' + str(month) + '_6m.gdbc.gz'
        QMO = rg.grid(name)
        QMO.Load()
        qmo = QMO.Data  # .flatten()
        qmo[np.isnan(qmo)] = 0

        cmd_remove = 'rm -f ' + TC_Contemp + 'qmo.txt'
        sp.call(cmd_remove, shell=True)
    
        if month == 1:
            tcq_mo_array = qmo
        else:
            tcq_mo_array = np.concatenate(([tcq_mo_array, qmo]), axis=0)
    
    tempname = TC_Contemp + 'TerraClimate_ROV_' + str(x) + '_Monthly.tif'
    CreateMultiGeoTiff(qannmo[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
    tempname = TC_Contemp + 'TerraClimate_Discharge_' + str(x) + '_Monthly.tif'
    CreateMultiGeoTiff(tcq_mo_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

    # Delete 2m30s GeoTIFF files
    cmd_remove = 'rm -f ' + TC_Contemp + 'q_mo2m30.tif'
    #sp.call(cmd_remove, shell=True)
    cmd_remove = 'rm -f ' + TC_Contemp + 'q_annual2m30.tif'
    #sp.call(cmd_remove, shell=True)

    # Remove gdbc files no longer needed
    cmd_remove = 'rm -f ' + TC_Contemp + '*.gdbc.gz'
    sp.call(cmd_remove, shell=True)

tempname = TC_Contemp + 'TerraClimate_ROV_Monthly.tif'
CreateMultiGeoTiff(tcrov_mo_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)


## 4C runs

In [ ]:
print("Resample TC to 6min...")
    
for x in range(StartYear, EndYear):
        
    # Calculate TC runoff from monthly AET and PPT xarrrays and save to GeoTIFFs for resampling
    aet_mo = ds_aet['aet'].values
    ppt_mo = ds_ppt['ppt'].values
    qmo = ppt_mo - aet_mo
    qmo = np.where(qmo < 0, 0, qmo)
    tempname = TC_ClimFut + 'q_mo2m30.tif'
    CreateMultiGeoTiff(qmo[:, :, :], tempname, geodriver, nodata, geotr2m30, geoproj, gdal.GDT_Float32)
    
    # Resample 2m30s monthly rasters to 6m resolution with nearest neighbor
    infn = TC_ClimFut + 'q_mo2m30.tif'
    outfn = TC_ClimFut + '4C/TerraClimate_RO_' + str(x) + '_6m.tif'
    xres=0.1
    yres=0.1
    resample_alg = 'average'
    options = gdal.WarpOptions(options=['tr'], xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = gdal.Warp(outfn, infn, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = None
    
    ds = gdal.Open(str(outfn))
    #qannmo = np.array(ds.GetRasterBand(1).ReadAsArray())
    qannmo = ds.ReadAsArray()
    qannmo[np.isnan(qannmo)] = 0
    qannmo = qannmo[:, :-n, :]
    qannmo = np.where(qannmo < 0, 0.00, qannmo)
    
    cellarea6m3D = np.resize(cellarea6m, (1,nrows6m,ncols6m))
    qannmo = qannmo * cellarea6m3D * (0.000380257053768347)
    
    if x == StartYear:
        tcrov_mo_array = qannmo
    else:
        tcrov_mo_array = np.concatenate(([tcrov_mo_array, qannmo]), axis=0)

    for y in range(0, 12):
        month = y + 1
        print(str(x) + "-" + str(month) + " ", end='')
        np.savetxt(TC_ClimFut + 'qmo.txt', qannmo[y, :, :], fmt='%1.9f', delimiter='  ', header=str(head6m), comments='')
        cmd = TC_ClimFut + 'netAcc6m_moClimFut.sh'
        sp.call(cmd, shell=True)
        cmd = 'mv ' + TC_ClimFut + 'Discharge_mo.gdbc.gz ' + TC_ClimFut + '4C/TerraClimate_Discharge_4C_' + str(x) + '-' + str(month) + '_6m.gdbc.gz'
        sp.call(cmd, shell=True)
    
        name = str(TC_ClimFut) + '4C/TerraClimate_Discharge_4C_' + str(x) + '-' + str(month) + '_6m.gdbc.gz'
        QMO = rg.grid(name)
        QMO.Load()
        qmo = QMO.Data  # .flatten()
        qmo[np.isnan(qmo)] = 0

        cmd_remove = 'rm -f ' + TC_ClimFut + 'qmo.txt'
        sp.call(cmd_remove, shell=True)
    
        if month == 1:
            tcq_mo_array = qmo
        else:
            tcq_mo_array = np.concatenate(([tcq_mo_array, qmo]), axis=0)
    
    tempname = TC_ClimFut + '4C/TerraClimate_ROV_4C_' + str(x) + '_Monthly.tif'
    CreateMultiGeoTiff(qannmo[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
    tempname = TC_ClimFut + '4C/TerraClimate_Discharge_4C_' + str(x) + '_Monthly.tif'
    CreateMultiGeoTiff(tcq_mo_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

    # Delete 2m30s GeoTIFF files
    cmd_remove = 'rm -f ' + TC_ClimFut + 'q_mo2m30.tif'
    #sp.call(cmd_remove, shell=True)
    cmd_remove = 'rm -f ' + TC_ClimFut + 'q_annual2m30.tif'
    #sp.call(cmd_remove, shell=True)

    # Remove gdbc files no longer needed
    cmd_remove = 'rm -f ' + TC_ClimFut + '4C/*.gdbc.gz'
    sp.call(cmd_remove, shell=True)

tempname = TC_ClimFut + '4C/TerraClimate_ROV_4C_Monthly.tif'
CreateMultiGeoTiff(tcrov_mo_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)


## 2C runs

In [ ]:
print("Resample TC to 6min...")
    
for x in range(StartYear, EndYear):
        
    # Calculate TC runoff from monthly AET and PPT xarrrays and save to GeoTIFFs for resampling
    aet_mo = ds_aet['aet'].values
    ppt_mo = ds_ppt['ppt'].values
    qmo = ppt_mo - aet_mo
    qmo = np.where(qmo < 0, 0, qmo)
    tempname = TC_ClimFut + 'q_mo2m30.tif'
    CreateMultiGeoTiff(qmo[:, :, :], tempname, geodriver, nodata, geotr2m30, geoproj, gdal.GDT_Float32)
    
    # Resample 2m30s monthly rasters to 6m resolution with nearest neighbor
    infn = TC_ClimFut + 'q_mo2m30.tif'
    outfn = TC_ClimFut + '2C/TerraClimate_RO_' + str(x) + '_6m.tif'
    xres=0.1
    yres=0.1
    resample_alg = 'average'
    options = gdal.WarpOptions(options=['tr'], xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = gdal.Warp(outfn, infn, xRes=xres, yRes=yres, resampleAlg=resample_alg)
    ds = None
    
    ds = gdal.Open(str(outfn))
    #qannmo = np.array(ds.GetRasterBand(1).ReadAsArray())
    qannmo = ds.ReadAsArray()
    qannmo[np.isnan(qannmo)] = 0
    qannmo = qannmo[:, :-n, :]
    qannmo = np.where(qannmo < 0, 0.00, qannmo)
    
    cellarea6m3D = np.resize(cellarea6m, (1,nrows6m,ncols6m))
    qannmo = qannmo * cellarea6m3D * (0.000380257053768347)
    
    if x == StartYear:
        tcrov_mo_array = qannmo
    else:
        tcrov_mo_array = np.concatenate(([tcrov_mo_array, qannmo]), axis=0)

    for y in range(0, 12):
        month = y + 1
        print(str(x) + "-" + str(month) + " ", end='')
        np.savetxt(TC_ClimFut + 'qmo.txt', qannmo[y, :, :], fmt='%1.9f', delimiter='  ', header=str(head6m), comments='')
        cmd = TC_ClimFut + 'netAcc6m_moClimFut.sh'
        sp.call(cmd, shell=True)
        cmd = 'mv ' + TC_ClimFut + 'Discharge_mo.gdbc.gz ' + TC_ClimFut + '2C/TerraClimate_Discharge_2C_' + str(x) + '-' + str(month) + '_6m.gdbc.gz'
        sp.call(cmd, shell=True)
    
        name = str(TC_ClimFut) + '2C/TerraClimate_Discharge_2C_' + str(x) + '-' + str(month) + '_6m.gdbc.gz'
        QMO = rg.grid(name)
        QMO.Load()
        qmo = QMO.Data  # .flatten()
        qmo[np.isnan(qmo)] = 0

        cmd_remove = 'rm -f ' + TC_ClimFut + 'qmo.txt'
        sp.call(cmd_remove, shell=True)
    
        if month == 1:
            tcq_mo_array = qmo
        else:
            tcq_mo_array = np.concatenate(([tcq_mo_array, qmo]), axis=0)
    
    tempname = TC_ClimFut + '2C/TerraClimate_ROV_2C_' + str(x) + '_Monthly.tif'
    CreateMultiGeoTiff(qannmo[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
    tempname = TC_ClimFut + '2C/TerraClimate_Discharge_2C_' + str(x) + '_Monthly.tif'
    CreateMultiGeoTiff(tcq_mo_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)

    # Delete 2m30s GeoTIFF files
    cmd_remove = 'rm -f ' + TC_ClimFut + 'q_mo2m30.tif'
    #sp.call(cmd_remove, shell=True)
    cmd_remove = 'rm -f ' + TC_ClimFut + 'q_annual2m30.tif'
    #sp.call(cmd_remove, shell=True)

    # Remove gdbc files no longer needed
    cmd_remove = 'rm -f ' + TC_ClimFut + '2C/*.gdbc.gz'
    sp.call(cmd_remove, shell=True)

tempname = TC_ClimFut + '2C/TerraClimate_ROV_2C_Monthly.tif'
CreateMultiGeoTiff(tcrov_mo_array[:, :, :], tempname, geodriver, nodata, geotr6m, geoproj, gdal.GDT_Float32)
